In [39]:
import numpy as np , math
from skimage import io
from CarDetection_HOG import display , get_hist_grid , Block_Normalization
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
from skimage.transform import resize

In [2]:
x = np.zeros((17584,3528) , dtype=np.float)
y = np.zeros((17584) , dtype=np.float)

img_index = 1
#Reading Vehicles & Non-Vehicles images:
for i in range(0 , 8792 , 2):
    img = io.imread("D:\\CESS Courses\\Brain ADAS\\Computer Vision\\Vehicles\\"+ "v_ (" + str(img_index) + ").png")
    x[i,:] = Block_Normalization(get_hist_grid(img , 8))
    y[i] = 1.0
    
    img = io.imread("D:\\CESS Courses\\Brain ADAS\\Computer Vision\\Non-Vehicles\\"+ "nv_ (" + str(img_index) + ").png")
    x[i+1,:] = Block_Normalization(get_hist_grid(img , 8))
    y[i+1] = 0.0
    
    img_index += 1

In [3]:
x_train , x_test , y_train , y_test = train_test_split(x , y, test_size = 0.20)
clf = SVC(kernel='linear')
clf.fit(x_train , y_train)
p = clf.predict(x_test)
print('Prediction ', p)
print('Accuracy ' , accuracy_score(y_test , p))

Prediction  [0. 0. 0. ... 0. 0. 0.]
Accuracy  0.978675007108331


In [4]:
def Classify(img):
    res = clf.predict(img)
    return res[0]

In [36]:
def Crop_Image(img):
    return img[img.shape[0]//2 : img.shape[0]]

In [45]:
def Slide_Window(img , window_size , step , scale):
    img = Crop_Image(img)
    img = resize(img, (scale, scale), anti_aliasing=True)
    h = ((img.shape[0] - window_size) + 1) // step
    w = ((img.shape[1] - window_size) + 1) // step
    
    R,C = img.shape[0] , img.shape[1]
    mat = np.zeros((h,w) , dtype = np.byte)
    window = np.zeros((1,3528) , dtype=np.float)
    i = -1
    
    for I in range(0 , R , step):
        if(R-I >= window_size):
            i += 1
            j = 0
            for J in range(0 , C , step):
                if(C-J >= window_size):
                    window[0,:] = Block_Normalization(get_hist_grid(img[I : I+window_size , J : J+window_size] , 8))
                    mat[i][j] = Classify(window)
                    
                    j += 1
        
                else:
                    break
        else:
            break
                              
    return mat

In [46]:
im = io.imread('x.jpeg')
qt = Slide_Window(im , 64 , 1 , 128)
print(qt)

C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
